In [27]:
import pulp

# Initialize the MILP problem
evacuation_problem = pulp.LpProblem("EvacuationOptimization", pulp.LpMinimize)

# Assuming we have a list of nodes, edges, cars, and other necessary parameters
# For example:
# nodes = [('node1', {'demand': 10, 'type': 'source'}), ('node2', {'demand': 0, 'type': 'sink'}), ...]
# edges = [(('node1', 'node2'), {'travel_time': 5, 'congestion': 1}), ...]
# cars = [{'location': 'node1', 'capacity': 4, 'available': True}, ...]
# max_time = some maximum time for the evacuation process
# Example data
nodes = {
    'S1': {'demand': 30, 'type': 'source'},
    'S2': {'demand': 20, 'type': 'source'},
    'S3': {'demand': 25, 'type': 'source'},
    'T1': {'demand': 0, 'type': 'sink'},
    'T2': {'demand': 0, 'type': 'sink'},
    'N1': {'demand': 0, 'type': 'intermediate'},
    'N2': {'demand': 0, 'type': 'intermediate'},
    'N3': {'demand': 0, 'type': 'intermediate'},
    'N4': {'demand': 0, 'type': 'intermediate'},
    'N5': {'demand': 0, 'type': 'intermediate'}
}

edges = {
    ('S1', 'N1'): {'travel_time': 5, 'congestion': 1},
    ('S2', 'N2'): {'travel_time': 3, 'congestion': 1},
    ('S3', 'N3'): {'travel_time': 4, 'congestion': 1},
    ('N1', 'N4'): {'travel_time': 2, 'congestion': 0.8},
    ('N2', 'N4'): {'travel_time': 2, 'congestion': 0.8},
    ('N3', 'N5'): {'travel_time': 2, 'congestion': 0.9},
    ('N4', 'T1'): {'travel_time': 5, 'congestion': 1},
    ('N5', 'T2'): {'travel_time': 5, 'congestion': 1}
}

cars = [
    {'location': 'S1', 'capacity': 4, 'available': True},
    {'location': 'S2', 'capacity': 4, 'available': True},
    {'location': 'S3', 'capacity': 4, 'available': True},
    {'location': 'N1', 'capacity': 4, 'available': True}
]

max_time = 100  # The time horizon for the optimization

In [28]:
# Define decision variables

# x_i_j_t = Number of cars sent from node i to node j at time t
# y_i_t =   Number of people evacuated from node i at time t
# z_i = Binary variable indicated whether or not a car is available at node i
x_vars = {(i, j, t): pulp.LpVariable(f'x_{i}_{j}_{t}', lowBound=0, cat='Integer')
          for i in nodes for j in nodes if (i, j) in edges for t in range(max_time)}
y_vars = {(i, t): pulp.LpVariable(f'y_{i}_{t}', lowBound=0, cat='Integer')
          for i in nodes for t in range(max_time)}
z_vars = {i: pulp.LpVariable(f'z_{i}', cat='Binary')
          for i, _ in nodes}

In [29]:
# Define the objective function

# Minimize T, the total time to evacuate all people from the source nodes to the sink nodes.
evacuation_problem += pulp.lpSum([x_vars[i, j, t] * edges[i, j]['travel_time'] * edges[i, j]['congestion']
                                  for i, j in edges for t in range(max_time)]), "TotalEvacuationTime"

In [30]:
# Constraints for the MILP problem

# Constraint for flow conservation at each node
for node in nodes:
    for t in range(max_time):
        # For source nodes, the outflow is the number of people evacuated
        if nodes[node]['type'] == 'source':
            evacuation_problem += (
                pulp.lpSum([x_vars[(node, j, t)] for j in nodes if (node, j) in edges]) == y_vars[(node, t)],
                f"FlowConservation_out_{node}_{t}"
            )
        # For sink nodes, the inflow is the number of people arriving
        elif nodes[node]['type'] == 'sink':
            evacuation_problem += (
                pulp.lpSum([x_vars[(i, node, t)] for i in nodes if (i, node) in edges]) == y_vars[(node, t)],
                f"FlowConservation_in_{node}_{t}"
            )
        # For intermediate nodes, inflow equals outflow
        else:
            evacuation_problem += (
                pulp.lpSum([x_vars[(i, node, t)] for i in nodes if (i, node) in edges]) -
                pulp.lpSum([x_vars[(node, j, t)] for j in nodes if (node, j) in edges]) == 0,
                f"FlowConservation_intermediate_{node}_{t}"
            )

# Constraint for meeting the demand at source nodes
for node in nodes:
    if nodes[node]['type'] == 'source':
        evacuation_problem += (
            pulp.lpSum([y_vars[(node, t)] for t in range(max_time)]) == nodes[node]['demand'],
            f"DemandConstraint_{node}"
        )

# Constraint for car capacities
for car in cars:
    location = car['location']
    capacity = car['capacity']
    for t in range(max_time):
        evacuation_problem += (
            pulp.lpSum([x_vars[(location, j, t)] for j in nodes if (location, j) in edges]) <= capacity,
            f"CarCapacity_{location}_{t}"
        )

# Constraint for car availability
for node in nodes:
    for t in range(max_time):
        evacuation_problem += (
            pulp.lpSum([x_vars[(i, node, t)] for i in nodes if (i, node) in edges]) +
            pulp.lpSum([x_vars[(node, j, t)] for j in nodes if (node, j) in edges]) <=
            (z_vars[node] * sum(car['capacity'] for car in cars if car['location'] == node)),
            f"CarAvailability_{node}_{t}"
        )

# Constraint for traffic congestion affecting travel time
for (i, j), edge_data in edges.items():
    for t in range(max_time):
        evacuation_problem += (
            x_vars[(i, j, t)] <= edge_data['congestion'] * (z_vars[i] * sum(car['capacity'] for car in cars if car['location'] == i)),
            f"TrafficConstraint_{i}_{j}_{t}"
        )


KeyError: 'S1'

In [31]:
# Solve the problem using the default solver
evacuation_problem.solve()

# Check the status of the solution
print("Status:", pulp.LpStatus[evacuation_problem.status])

# Output the optimal schedule
for variable in evacuation_problem.variables():
    print(f"{variable.name} = {variable.varValue}")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/xs/5nydm67x3fj41732cnf_d1nw0000gq/T/7727e07b5d1f4359bcc16036eb2230a4-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/xs/5nydm67x3fj41732cnf_d1nw0000gq/T/7727e07b5d1f4359bcc16036eb2230a4-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 1408 COLUMNS
At line 7609 RHS
At line 9013 BOUNDS
At line 10314 ENDATA
Problem MODEL has 1403 rows, 1300 columns and 2800 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 810 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from 810 to -1.79769e+308
Probing was tried 0 times 